In [52]:
import sqlite3
import pandas as pd

con = sqlite3.connect('C:\\Users\\Mekakris\\Documents\\AC\\Prautise\\deeplearnig.sqlite')

data = pd.read_sql_query("select * from PractiseCleanedReviews where score!=3",con)

print(data.shape)


def parition(x):
    if x=='positive':
        return 1
    return 0

actualscore=data['Score']

positiveNegative=actualscore.map(parition)

data['Score']=positiveNegative

(6991, 11)


# Apply tfidf and get the standard data for the model

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import numpy as np
from sklearn.preprocessing import StandardScaler


count_vect = TfidfVectorizer(ngram_range=(1,2),max_features=5000)# -> this is for normal text vector\

labels=data['Score'].values

final=count_vect.fit_transform(data['Text'].values)
standardized_data = StandardScaler(with_mean=False).fit_transform(final)


# Keras with TF-IDF

In [60]:
print(standardized_data.shape)
print(labels.shape)

(6991, 5000)
(6991,)


# Model Parameters

In [79]:
from keras.models import Sequential
from keras.layers import Activation,Dense,Dropout


model=Sequential()

# 1st Layer
model.add(Dense(5000,input_shape=(5000,)))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))

# 2nd Layer
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.5))
          
# 3rd Layer
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 5000)              25005000  
_________________________________________________________________
activation_65 (Activation)   (None, 5000)              0         
_________________________________________________________________
dropout_44 (Dropout)         (None, 5000)              0         
_________________________________________________________________
dense_68 (Dense)             (None, 1000)              5001000   
_________________________________________________________________
activation_66 (Activation)   (None, 1000)              0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_69 (Dense)             (None, 2)                 2002      
__________

In [80]:
from keras.utils import np_utils 
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

history=model.fit(standardized_data,np_utils.to_categorical(labels, 2) ,batch_size=1000,epochs=5,verbose=1,validation_split=0.1)

Train on 6291 samples, validate on 700 samples
Epoch 1/5
6291/6291 [==============================] - 25s 4ms/step - loss: 0.7553 - acc: 0.5144 - val_loss: 0.6878 - val_acc: 0.5671
Epoch 2/5
6291/6291 [==============================] - 28s 4ms/step - loss: 0.7026 - acc: 0.5204 - val_loss: 0.6877 - val_acc: 0.5771
Epoch 3/5
6291/6291 [==============================] - 23s 4ms/step - loss: 0.6973 - acc: 0.5260 - val_loss: 0.6872 - val_acc: 0.5771
Epoch 4/5
6291/6291 [==============================] - 26s 4ms/step - loss: 0.6948 - acc: 0.5274 - val_loss: 0.6876 - val_acc: 0.5771
Epoch 5/5
6291/6291 [==============================] - 20s 3ms/step - loss: 0.6926 - acc: 0.5311 - val_loss: 0.6876 - val_acc: 0.5771
